<a href="https://colab.research.google.com/github/ShounakDas101/AIML_Hari/blob/main/TopGunGATHari.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import pyarrow.parquet as pq

# topgun_file ='/content/drive/MyDrive/data/raw/output_2048_lines.parquet'
# # Read the Parquet file into a PyArrow Table
# parquet_table = pq.read_table(topgun_file)
# # Convert the PyArrow Table to a pandas DataFrame
# data_frame = parquet_table.to_pandas()
# # Get the keys (column names) of the DataFrame
# keys = data_frame.columns.tolist()  # Convert the column names to a list
# print(keys)
# # data_frame['m']
# print(data_frame['m'],data_frame['iphi'],data_frame['pt'],data_frame['ieta'])

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.1.0+cu121
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Device cpu


In [ ]:
import torch_geometric
import numpy as np
from torch_geometric.utils import get_laplacian, to_scipy_sparse_matrix, to_dense_adj, to_undirected


def eigvec_normalizer(EigVecs, EigVals, normalization="L2", eps=1e-12):
    """
    Implement different eigenvector normalizations.
    """

    EigVals = EigVals.unsqueeze(0)

    if normalization == "L1":
        # L1 normalization: eigvec / sum(abs(eigvec))
        denom = EigVecs.norm(p=1, dim=0, keepdim=True)

    elif normalization == "L2":
        # L2 normalization: eigvec / sqrt(sum(eigvec^2))
        denom = EigVecs.norm(p=2, dim=0, keepdim=True)

    elif normalization == "abs-max":
        # AbsMax normalization: eigvec / max|eigvec|
        denom = torch.max(EigVecs.abs(), dim=0, keepdim=True).values

    elif normalization == "wavelength":
        # AbsMax normalization, followed by wavelength multiplication:
        # eigvec * pi / (2 * max|eigvec| * sqrt(eigval))
        denom = torch.max(EigVecs.abs(), dim=0, keepdim=True).values
        eigval_denom = torch.sqrt(EigVals)
        eigval_denom[EigVals < eps] = 1  # Problem with eigval = 0
        denom = denom * eigval_denom * 2 / np.pi

    elif normalization == "wavelength-asin":
        # AbsMax normalization, followed by arcsin and wavelength multiplication:
        # arcsin(eigvec / max|eigvec|)  /  sqrt(eigval)
        denom_temp = torch.max(EigVecs.abs(), dim=0, keepdim=True).values.clamp_min(eps).expand_as(EigVecs)
        EigVecs = torch.asin(EigVecs / denom_temp)
        eigval_denom = torch.sqrt(EigVals)
        eigval_denom[EigVals < eps] = 1  # Problem with eigval = 0
        denom = eigval_denom

    elif normalization == "wavelength-soft":
        # AbsSoftmax normalization, followed by wavelength multiplication:
        # eigvec / (softmax|eigvec| * sqrt(eigval))
        denom = (F.softmax(EigVecs.abs(), dim=0) * EigVecs.abs()).sum(dim=0, keepdim=True)
        eigval_denom = torch.sqrt(EigVals)
        eigval_denom[EigVals < eps] = 1  # Problem with eigval = 0
        denom = denom * eigval_denom

    else:
        raise ValueError(f"Unsupported normalization `{normalization}`")

    denom = denom.clamp_min(eps).expand_as(EigVecs)
    EigVecs = EigVecs / denom

    return EigVecs

def get_lap_decomp_stats(evals, evects, max_freqs, eigvec_norm='L2'):

    N = len(evals)  # Number of nodes, including disconnected nodes.

    # Keep up to the maximum desired number of frequencies.
    idx = evals.argsort()[:max_freqs]
    evals, evects = evals[idx], np.real(evects[:, idx])
    evals = torch.from_numpy(np.real(evals)).clamp_min(0)

    # Normalize and pad eigen vectors.
    evects = torch.from_numpy(evects).float()
    evects = eigvec_normalizer(evects, evals, normalization=eigvec_norm)
    if N < max_freqs:
        EigVecs = F.pad(evects, (0, max_freqs - N), value=float('nan'))
    else:
        EigVecs = evects

    # Pad and save eigenvalues.
    if N < max_freqs:
        EigVals = F.pad(evals, (0, max_freqs - N), value=float('nan')).unsqueeze(0)
    else:
        EigVals = evals.unsqueeze(0)
    EigVals = EigVals.repeat(N, 1).unsqueeze(2)

    return EigVals, EigVecs

def compute_enc_transform(x, edge_index, transform_flags):

    N = x.shape[0]

    to_return = {}

    if transform_flags["LapPE"]:
        undir_edge_idx = to_undirected(edge_index, num_nodes  = N)
        L = to_scipy_sparse_matrix(
            *get_laplacian(undir_edge_idx, normalization=transform_flags["LapPEnorm"], num_nodes=N)
        )
        evals, evects = np.linalg.eigh(L.toarray())
        max_freqs = transform_flags["LapPEmax_freq"]
        eigvec_norm = transform_flags["LapPEeig_norm"]

        EigVals, EigVecs = get_lap_decomp_stats(
            evals=evals, evects=evects,
            max_freqs=max_freqs,
            eigvec_norm=eigvec_norm
        )

        to_return['eigvals'] = EigVals
        to_return['eigvecs'] = EigVecs
    return to_return



In [ ]:
'''
def positional_encoding(data, pe_scales):
    pe_cos = torch.cat([torch.cos(2**i * np.pi * torch.as_tensor(data))
                       for i in range(pe_scales)], dim=1)
    pe_sin = torch.cat([torch.sin(2**i * np.pi * torch.as_tensor(data))
                       for i in range(pe_scales)], dim=1)

    output= torch.cat([torch.as_tensor(data), pe_cos, pe_sin], dim=1)
    return output
'''

'\ndef positional_encoding(data, pe_scales):\n    pe_cos = torch.cat([torch.cos(2**i * np.pi * torch.as_tensor(data))\n                       for i in range(pe_scales)], dim=1)\n    pe_sin = torch.cat([torch.sin(2**i * np.pi * torch.as_tensor(data))\n                       for i in range(pe_scales)], dim=1)\n\n    output= torch.cat([torch.as_tensor(data), pe_cos, pe_sin], dim=1)\n    return output\n'

In [ ]:
def normalize_x(x):
    x = x - np.array([0.01037084, 0.0103173, 0.01052679, 0.01034378, 0.01097225, 0.01024814, 0.01037642, 0.01058754])
    x = x / np.array([10.278656283775618, 7.64753320751208, 16.912319597559645, 9.005579923580713, 21.367327333103688, 7.489890622699373, 12.977402491253788, 24.50774893130742])

    return x

In [ ]:
import os
import pyarrow.parquet as pq

hcal_scale  = 1
ecal_scale  = 0.1
pt_scale    = 0.01
dz_scale    = 0.05
d0_scale    = 0.1
m0_scale    = 85
m1_scale    = 415
p0_scale = 400
p1_scale = 600

def points_all_channels(X_jets, suppression_thresh):
    idx = np.where(abs(X_jets).sum(axis=0) > suppression_thresh)
    pos = np.array(idx).T / X_jets.shape[1]
    x = X_jets[:, idx[0], idx[1]].T

    return x, pos

class PointCloudFromParquetDataset(torch.utils.data.Dataset):
    def __init__(self,data_dir,save_data,id,filename,
                    transform_flags,suppresion_thresh,k,min_mass,max_mass,num_bins):
        super().__init__()

        self.id = id
        self.file = pq.ParquetFile(filename)
        self.root_dir = data_dir
        self.save_data = save_data
        self.transform_flags = transform_flags
        self.suppression_thresh = suppresion_thresh
        self.k = k
        self.min_mass=min_mass
        self.max_mass=max_mass
        self.num_bins=num_bins

        bin_size=(max_mass-min_mass)/num_bins
        self.bins=[min_mass + i*bin_size for i in range(num_bins)]

    def __getitem__(self, idx, ):
        row = self.file.read_row_group(idx).to_pydict()
        arr = np.array(row['X_jet'][0])
        x, pos = points_all_channels(arr, self.suppression_thresh)
        x = normalize_x(x)
        pt = row['pt'][0]
        ieta = row['ieta'][0]
        iphi = row['iphi'][0]
        m = row['m'][0]
        m=m-m0_scale
        m=m/m1_scale
        m_class= (-1)
        x[:, 0] *= pt_scale
        x[:, 1] *= dz_scale
        x[:, 2] *= d0_scale
        x[:, 3] *= ecal_scale
        x[:, 4] *= hcal_scale
        pt = (pt - p0_scale) / p1_scale
        iphi = iphi / 360.
        ieta = ieta / 140.

                        # High value suppression
        x[:, 1][x[:, 1] < -1] = 0
        x[:, 1][x[:, 1] > 1] = 0
        x[:, 2][x[:, 2] < -1] = 0
        x[:, 2][x[:, 2] > 1] = 0

                        # Zero suppression
        x[:, 0][x[:, 0] < 1e-2] = 0.
        x[:, 3][x[:, 3] < 1e-2] = 0.
        x[:, 4][x[:, 4] < 1e-2] = 0.
        for it,bin_start in enumerate(self.bins):
            if bin_start > m:
                m_class= it -1
                break
        if m_class == -1:
            m_class=self.num_bins -1
        y_class=m_class
        x = np.concatenate([x, pos], axis=1)
        pos = torch.as_tensor(pos, dtype=torch.float)
        x = torch.as_tensor(x)
        edge_index = torch_geometric.nn.knn_graph(x=pos, k = self.k, num_workers=0)
        transforms = compute_enc_transform(x, edge_index, self.transform_flags)
        print(transforms['eigvecs'].shape)
        print(transforms['eigvals'].shape)
        x = torch.cat([x, transforms['eigvecs'], transforms['eigvals'].squeeze(-1)], dim=-1)
        print(x.shape)
        data = torch_geometric.data.Data(
            pos=pos.float(),x=x.float(),
            pt=torch.as_tensor(pt).unsqueeze(-1),
            ieta=torch.as_tensor(ieta).unsqueeze(-1),
            iphi=torch.as_tensor(iphi).unsqueeze(-1),
            y=torch.as_tensor(m),
            y_class=y_class
        )
        if self.save_data:
            torch.save(data, os.path.join(self.save_data, f'{self.id}_{idx}.pt'))

        return data

    def __len__(self):
        return self.file.num_row_groups



In [ ]:
x=PointCloudFromParquetDataset(
                    data_dir = '/content/drive/MyDrive/data/',
                    save_data = False,
                    id = 0,
                    filename = '/content/drive/MyDrive/data/raw/top_gun_opendata_1.parquet',
                    transform_flags = {"LapPE": True,"LapPEnorm": "sym","LapPEmax_freq": 10,"LapPEeig_norm": "L2","RWSE": False,"RWSEkernel_times": [2, 3, 5, 7, 10]},
                    suppresion_thresh=1e-3,
                    k = 20,min_mass=0,max_mass=1,num_bins=10
                )

In [ ]:
len(x)

150165

In [ ]:
import torch
import glob
import os
import random


from tqdm.auto import tqdm

class TopGunPreprocessor():
    def __init__(self,data_dir,num_files,test_ratio,val_ratio,transform_flags,min_threshold,k,min_mass=0,max_mass=40,num_bins=4):
        self.data_dir=data_dir
        self.num_files=num_files
        self.test_ratio=test_ratio
        self.val_ratio=val_ratio
        self.transform_flags=transform_flags
        self.min_threshold=min_threshold
        self.k=k
        self.min_mass=min_mass
        self.max_mass=max_mass
        self.num_bins=num_bins
        self.preprocess()

    def preprocess(self):
        paths = list(glob.glob(os.path.join(self.data_dir, "raw", "*.parquet")))

        dsets = []
        for it,path in enumerate(tqdm(paths[0:self.num_files])):
            dsets.append(
                PointCloudFromParquetDataset(
                    data_dir = self.data_dir,
                    save_data = os.path.join(self.data_dir, "saved"),
                    id = it,
                    filename = path,
                    transform_flags = self.transform_flags,
                    suppresion_thresh=self.min_threshold,
                    k = self.k,min_mass=self.min_mass,max_mass=self.max_mass,num_bins=self.num_bins
                )
            )

        combined_dset = torch.utils.data.ConcatDataset(dsets)

        sampled_data_size = int(len(combined_dset) * 0.005)

        random_indices = random.sample(range(len(combined_dset)), sampled_data_size)

        combined_dset = torch.utils.data.Subset(combined_dset, random_indices)

        val_size = int(len(combined_dset) * self.val_ratio)
        test_size = int(len(combined_dset) * self.test_ratio)
        train_size = len(combined_dset) - val_size - test_size

        train_dset, val_dset, test_dset = torch.utils.data.random_split(
            combined_dset,
            [train_size, val_size, test_size],
            generator=torch.Generator().manual_seed(42),
        )

        self.train_dataset = train_dset
        self.val_dataset = val_dset
        self.test_dataset = test_dset

In [ ]:
dset = TopGunPreprocessor(data_dir='/content/drive/MyDrive/data/',num_files=1,test_ratio=.2,val_ratio=.2,transform_flags={"LapPE": True,"LapPEnorm": "sym","LapPEmax_freq": 10,"LapPEeig_norm": "L2","RWSE": False,"RWSEkernel_times": [2, 3, 5, 7, 10]},min_threshold=1e-3,k = 20,min_mass=0,max_mass=1,num_bins=10)

In [ ]:
train_dataset=dset.train_dataset
val_dataset = dset.val_dataset
test_dataset = dset.test_dataset

In [ ]:
print(len(test_dataset))
print(len(val_dataset))

In [ ]:
# !pip uninstall torch-cluster -y
# !pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html

In [ ]:
# !pip install torch-cluster

In [ ]:
for i in range(5):
    print(test_dataset[i])

In [ ]:
loader_type = torch_geometric.loader.DataLoader
train_loader = loader_type( train_dataset, shuffle=True, batch_size=128, pin_memory=True, num_workers=0,drop_last=True)
val_loader = loader_type( val_dataset, shuffle=True, batch_size=128, pin_memory=True, num_workers=0,drop_last=True)
test_loader = loader_type( test_dataset, shuffle=True, batch_size=128, pin_memory=True, num_workers=0,drop_last=True)

In [ ]:
for batch in train_loader:
    print(batch)
    break

In [ ]:

# clearing cuda cache memory
import gc
torch.cuda.empty_cache()
gc.collect()


In [ ]:
import torch.nn as nn

class SimpleGAT(nn.Module):
    def __init__(self,  x_size, edge_feat='none', k=7, use_pe=False, pe_scales=0):
        super().__init__()
        self.k = k
        self.edge_feat = edge_feat
        # self.args = args
        if self.edge_feat == 'none':
            edge_dim = None

        self.gat_conv_1 = torch_geometric.nn.GATv2Conv(
            in_channels=x_size if not use_pe else x_size * (pe_scales * 2 + 1),
            out_channels=16,
            heads=4,
            edge_dim=edge_dim
        )
        self.bn_1 = torch_geometric.nn.BatchNorm(64)
        self.gat_conv_2 = torch_geometric.nn.GATv2Conv(
            in_channels=16 * 4,
            out_channels=32,
            heads=4,
            edge_dim=edge_dim
        )
        self.bn_2 = torch_geometric.nn.BatchNorm(128)
        self.act = torch.nn.ReLU()



    def forward(self, data):
        pos = data.pos
        batch = data.batch
        x = data.x

        edge_index = torch_geometric.nn.knn_graph(x=pos, k=self.k, batch=batch)
        if self.edge_feat == 'none':
            edge_attr = None

        else:
            raise NotImplementedError(f"Edge feat {self.edge_feat} is not implemented")

        x_out = self.act(self.bn_1(self.gat_conv_1(
            x, edge_index, edge_attr=edge_attr)))
        x_out = self.act(self.bn_2(self.gat_conv_2(
            x_out, edge_index, edge_attr=edge_attr)))

        x_out = torch_geometric.nn.global_mean_pool(x_out, batch)

        return x_out

    def __str__(self):
        """
        Model prints with number of trainable parameters
        """
        model_parameters = filter(lambda p: p.requires_grad, self.parameters())
        params = sum([np.prod(p.size()) for p in model_parameters])
        return super().__str__() + '\nTrainable parameters: {}'.format(params)

In [ ]:
class MLPStack(torch.nn.Module):
    def __init__(self, layers, bn=True, act=True, p=0):
        super().__init__()
        assert len(layers) > 1, "At least input and output channels must be provided"

        modules = []
        for i in range(1, len(layers)):
            modules.append(
                torch.nn.Linear(layers[i-1], layers[i])
            )
            modules.append(
                torch.nn.BatchNorm1d(layers[i]) if bn == True else torch.nn.Identity()
            )
            modules.append(
                torch.nn.SiLU() if bn == True else torch.nn.Identity()
            )
            modules.append(
                torch.nn.Dropout(p=p) if p != 0 else torch.nn.Identity()
            )

        self.mlp_stack = torch.nn.Sequential(*modules)

    def forward(self, x):
        return self.mlp_stack(x)

class RegressModel(nn.Module):
    def __init__(self, model, in_features, predict_bins=True, num_bins=10):
        super().__init__()
        self.model = model
        self.predict_bins = predict_bins

        self.out_mlp = MLPStack(
            [in_features + 3, in_features * 2, in_features * 2, in_features, in_features // 2],
            bn=True, act=True
        )

        self.out_regress = torch.nn.Linear(in_features//2, 1)

        if self.predict_bins:
            self.out_pred = torch.nn.Linear(in_features // 2, num_bins)

    def forward(self, data):
        return_dict = {}

        print(data[0].x,data[0].pos,data[0].pt,data[0].y,data[0].ieta)
        out = self.model(data)
        out = torch.cat(
            [out, data.pt.unsqueeze(-1), data.ieta.unsqueeze(-1), data.iphi.unsqueeze(-1)], dim=1
        )
        out = self.out_mlp(out)
        regress_out = self.out_regress(out)

        return_dict['regress'] = regress_out

        if self.predict_bins:
            pred_out = self.out_pred(out)
            return_dict['class'] = pred_out

        return return_dict

    def __str__(self):
        """
        Model prints with number of trainable parameters
        """
        model_parameters = filter(lambda p: p.requires_grad, self.parameters())
        params = sum([np.prod(p.size()) for p in model_parameters])
        return super().__str__() + '\nTrainable parameters: {}'.format(params)

In [ ]:
input_size = 30


model = SimpleGAT(x_size=input_size)
Final_model=RegressModel(model, in_features = 128, predict_bins=True, num_bins=10)
Final_model.to(device)

In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm

# Assuming Final_model, train_loader, and val_loader are defined
lr = 0.001  # Assuming lr is defined

criterion = torch.nn.MSELoss().to(device)
trainable_params = filter(lambda p: p.requires_grad, Final_model.parameters())
# # Convert the filter object to a list or another iterable type
# trainable_params_list = list(trainable_params)

# # Print the trainable parameters
# print(trainable_params_list)
optimizer = torch.optim.AdamW(trainable_params, lr=lr)

criterion_dict = {}
criterion_dict['regress'] = torch.nn.MSELoss()
#criterion_dict['class'] = torch.nn.BCEWithLogitsLoss()

for epoch in range(2):
    # Training
    Final_model.train()
    tqdm_iter = tqdm(train_loader, total=len(train_loader))
    tqdm_iter.set_description(f"Epoch {epoch} - Training")

    total_train_loss = 0.0
    true_preds, num_preds = 0., 0.
    for it, batch in enumerate(tqdm_iter):
        batch = batch.to(device)
        optimizer.zero_grad()
        m = batch.y
        out = Final_model(batch)

        loss_dict = {}
        loss = 0

        for i in range(len(batch)):
                #print("out",out['regress'][i],"batch_yclass",batch.y_class[i])
            if(torch.floor(10*out['regress'][i])==batch.y_class[i]):
                true_preds+=1
                #print("out",torch.floor(10*out['regress'][i])==batch.y_class[i])
            #print(torch.floor(out['regress']*10))
        num_preds += len(batch)
        m = m * m1_scale + m0_scale
        out['regress'] = out['regress'] * m1_scale + m0_scale
        loss_dict['regress'] = criterion(out['regress'], m.unsqueeze(-1))
        loss += loss_dict['regress']

        total_train_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_train_loss = total_train_loss / len(train_loader)
    print(f"Epoch {epoch} - Average Training Loss: {average_train_loss:.4f}")
    print("train acc",true_preds/num_preds)
    # Validation
    Final_model.eval()
    val_tqdm_iter = tqdm(val_loader, total=len(val_loader))
    val_tqdm_iter.set_description(f"Epoch {epoch} - Validation")

    total_val_loss = 0.0
    true_preds, num_preds = 0., 0.
    for it, batch in enumerate(val_tqdm_iter):
        with torch.no_grad():
            batch = batch.to(device)
            m = batch.y
            out = Final_model(batch)

            loss_dict = {}
            loss = 0

            for i in range(len(batch)):
                #print("out",out['regress'][i],"batch_yclass",batch.y_class[i])
                if(torch.floor(10*out['regress'][i])==batch.y_class[i]):
                    true_preds+=1
                #print("out",torch.floor(10*out['regress'][i])==batch.y_class[i])
            #print(torch.floor(out['regress']*10))
            num_preds += len(batch)


            # Scale both target and predicted values before calculating the loss
            m = m * m1_scale + m0_scale
            out['regress'] = out['regress'] * m1_scale + m0_scale

# Calculate the loss on scaled values
            loss_dict['regress'] = criterion(out['regress'], m.unsqueeze(-1))
            loss += loss_dict['regress']



            total_val_loss += loss.item()

    average_val_loss = total_val_loss / len(val_loader)
    print(f"Epoch {epoch} - Average Validation Loss: {average_val_loss:.4f}")
    print("Val acc",true_preds/num_preds)
    # Print or compute validation accuracy if applicable
    # Add your validation accuracy computation code here

    # Reset the model back to training mode
    Final_model.train()


In [ ]:
for i in range(1):
    Final_model.eval()
    val_tqdm_iter = tqdm(val_loader, total=len(val_loader))
    val_tqdm_iter.set_description(f"Epoch {epoch} - Validation")

    total_val_loss = 0.0
    true_preds, num_preds = 0., 0.
    for it, batch in enumerate(val_tqdm_iter):
        with torch.no_grad():
            batch = batch.to(device)
            m = batch.y
            out = Final_model(batch)

            loss_dict = {}
            loss = 0

            for i in range(len(batch)):
                print("out",out['regress'][i],"batch_yclass",batch.y_class[i])
                if(torch.floor(10*out['regress'][i])==batch.y_class[i]):
                    true_preds+=1
                #print("out",torch.floor(10*out['regress'][i])==batch.y_class[i])
            #print(torch.floor(out['regress']*10))
            num_preds += len(batch)


            # Scale both target and predicted values before calculating the loss
            m = m * m1_scale + m0_scale
            out['regress'] = out['regress'] * m1_scale + m0_scale

# Calculate the loss on scaled values
            loss_dict['regress'] = criterion(out['regress'], m.unsqueeze(-1))
            loss += loss_dict['regress']



            total_val_loss += loss.item()

    average_val_loss = total_val_loss / len(val_loader)
    print(f"Epoch {epoch} - Average Validation Loss: {average_val_loss:.4f}")
    print("Val acc",true_preds/num_preds)